# Unit Testing Primer

## For this primer, I'll go over several different types of unit testing.  I'll do some simple examples and work up to some more complex situations

### Let's start simple
### This simple function accepts a string and returns the number of characters in the string.  We need to test happy path and that the expected exception occurs when we input a bad datatype.

In [ ]:
def count_the_letters(inputstring):
    return len(inputstring)

print(count_the_letters("whatever")) #happy path
print(count_the_letters(""))
print(count_the_letters(343434)) #expect a TypeError exception

In [13]:
import unittest

class TestLetterCounting(unittest.TestCase):
    def test_happy_path(self): # Recall that this function must start with "test" to actually run the tests
        self.assertEqual(count_the_letters("manycharacterslongstring"),24)
        self.assertEqual(count_the_letters(""),0)
        self.assertEqual(count_the_letters("12345"),5)
    
    def test_type_error(self): # An assertRaises test only passes when the correct assert is raised
        with self.assertRaises(TypeError):
            count_the_letters(2345) #The function expects a string so an integer will raise a TypeError

        
if __name__ == '__main__':
    unittest.main(argv=[''],exit=False) #This is specific to Jupyter Notebooks
    # It would look like this in PyCharm: unittest.main()

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


### Not too bad.  However, just like everything with coding, users make it more difficult :)
### Let's mock some user inputs.  I'm going to adjust the function above to request two strings from the user, count the total number of characters in the two strings, and return the result

In [17]:
def count_the_letters_with_input():
    string_1 = input("Give me the first string: ")
    string_2 = input("Give me the second string: ")
    return (len(string_1) + len(string_2))

count_the_letters_with_input()

Give me the first string: 23455
Give me the second string: 232456


11

#### Part of our job is easier here because we don't have to deal with the TypeError.  No matter what a user puts in, Python will treat it like a string by default

#### First I'm going to just use a generic return value for all inputs.

In [25]:
import unittest
from unittest import mock

class TestLetterCountingWithInput(unittest.TestCase): 
    def test_same_input_every_time(self):
        with mock.patch('builtins.input', return_value="whatever"): # This method mocks the user input as "whatever" as many times as the underlying function calls it
            assert count_the_letters_with_input() == 16 # Function receives "whatever" and "whatever" which is 16 characters
        
        with mock.patch('builtins.input', return_value=""): 
            assert count_the_letters_with_input() == 0 # Function receives no characters both times
            
if __name__ == '__main__':    
    unittest.main(argv=[''],exit=False) #This is specific to Jupyter Notebooks
    # It would look like this in PyCharm: unittest.main()

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


#### Now let's be more specific about the inputs that are mocked

In [29]:
import unittest
from unittest import mock

class TestLetterCountingWithInput(unittest.TestCase): 
    def test_different_input_every_time(self):
        with mock.patch('builtins.input', side_effect=["word","otherword"]):
            assert count_the_letters_with_input() == 13 # Function receives "whatever" and "whatever" which is 16 characters
        
        with mock.patch('builtins.input', side_effect=["abunchoftext",""]): 
            assert count_the_letters_with_input() == 12 # Function receives no characters both times
            
if __name__ == '__main__':    
    unittest.main(argv=[''],exit=False) #This is specific to Jupyter Notebooks
    # It would look like this in PyCharm: unittest.main()

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


### Now I'll create a program that has multiple functions and test each of those functions.  It's important to remember that unit testing doesn't test programs. It tests functions.  There are times that you may need to adjust how your program is structured in order to make it testable

### My program is going to ask for multiple user inputs, confirm that those inputs are valid, and then adds those inputs to a list.

In [49]:
def gather_user_input():
    user_input = input("give me one of your inputs: ")
    if verify_user_input_integer(user_input) == "fail":
        return "fail"
    else:
        return int(user_input)

def verify_user_input_integer(check_user_input):
    try:
        int(check_user_input)
    except:
        return "fail"
    else:
        return int(check_user_input)

def how_many_inputs():
    result = verify_user_input_integer(input("Please enter the number of integers you want to store in a list: "))
    if result == "fail":
        return "fail"
    else:
        return result
    
def get_all_numbers(quantity):
    user_list = []
    good_input_count = 0
    while good_input_count < quantity:
        user_input_integer = gather_user_input()
        if user_input_integer == "fail":
            print("Your input was not an integer so this input will be skipped")
            continue
        else:
            user_list.append(user_input_integer)
            good_input_count += 1
    return user_list

if __name__ == '__main__':
    user_integers = []
    number_of_inputs = how_many_inputs()
    if number_of_inputs == "fail":
        print("You failed to give me a properly formatted quantity of strings")
    else:
        user_integers = get_all_numbers(number_of_inputs)
    print("your list of integers: ", user_integers)

Please enter the number of integers you want to store in a list: ewr
You failed to give me a properly formatted quantity of strings
your list of integers:  []


#### In order to test my program functions, I have to test each of the four functions with various inputs and ensure they have the correct output.

In [61]:
import unittest
from unittest import mock

class TestGatherUserInput(unittest.TestCase): 
    def test_good_input(self):
        with mock.patch('builtins.input', side_effect=["13"]):
            assert gather_user_input() == 13 
        with mock.patch('builtins.input', side_effect=["1"]):
            assert gather_user_input() == 1 
    def test_bad_input(self):
        with mock.patch('builtins.input', side_effect=["whatever"]):
            assert gather_user_input() == "fail" 
        with mock.patch('builtins.input', side_effect=[""]):
            assert gather_user_input() == "fail" 
        
class TestVerifyUserInputInteger(unittest.TestCase):
    def test_good_datatype(self):
        self.assertEqual(verify_user_input_integer("10"),10)
        self.assertEqual(verify_user_input_integer("25"),25)
        
    def test_bad_datatype(self):
        self.assertEqual(verify_user_input_integer("ere"),"fail")
        self.assertEqual(verify_user_input_integer(""),"fail")
            
class TestHowManyInputs(unittest.TestCase):
    def test_good_num_input(self):
        with mock.patch('builtins.input', side_effect=["13"]):
            assert how_many_inputs() == 13 
        with mock.patch('builtins.input', side_effect=["1"]):
            assert how_many_inputs() == 1 
    def test_bad_num_input(self):
        with mock.patch('builtins.input', side_effect=["whatever"]):
            assert how_many_inputs() == "fail" 
        with mock.patch('builtins.input', side_effect=[""]):
            assert how_many_inputs() == "fail" 
            
class TestGetAllNumbers(unittest.TestCase):
    def test_good_user_list(self):
        with mock.patch('builtins.input', side_effect=[1,2,3]):
            assert get_all_numbers(3) == [1,2,3]
        with mock.patch('builtins.input', side_effect=[1,2,3,4,5,6,7,8]):
            assert get_all_numbers(8) == [1,2,3,4,5,6,7,8]
    def test_bad_user_list(self):
        with mock.patch('builtins.input', side_effect=["what",3,"ever",4,5]):
            assert get_all_numbers(3) == [3,4,5]
        with mock.patch('builtins.input', side_effect=["what",3,"ever",4,"bad","input","from",5]):
            assert get_all_numbers(3) == [3,4,5]
            
if __name__ == '__main__':    
    unittest.main(argv=[''],exit=False) #This is specific to Jupyter Notebooks
    # It would look like this in PyCharm: unittest.main()

...........

Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped
Your input was not an integer so this input will be skipped



----------------------------------------------------------------------
Ran 11 tests in 0.011s

OK


#### Now I have tested all four of my functions as independently as possible both through happy path and expected issues.  Note again that I'm not testing main anywhere in the unit testing.  Main is not tested in unit testing.